In [33]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import time
import textrazor
import pandas as pd
import numpy as np

In [2]:
from pymongo import MongoClient
import numpy as np

MONGO_PASSWORD = os.environ['MONGO_USER_PASSWORD']
CROWD_TANGLE = os.environ['CROWD_TANGLE_API_TOKEN']

config = {
  'host': '18.216.52.13:27017',
  'username': 'mongo_user',
  'password': MONGO_PASSWORD,
  'authSource': 'reddit_climate_news'
}

db = MongoClient(**config).reddit_climate_news

In [19]:
df = pd.DataFrame.from_records(db.climate_submissions.find())

In [23]:
valid_resp = df[df.reddit_crowd_tangle.map(lambda x: x.get('status') == 200)]

In [21]:
valid_resp.reddit_crowd_tangle[0]

{'posts': {'totals': {'share_count': 0,
   'comment_count': 45,
   'like_count': 104,
   'retweet_count': 0,
   'love_count': 0,
   'haha_count': 0,
   'wow_count': 0,
   'sad_count': 0,
   'angry_count': 0,
   'thankful_count': 0,
   'num_users': 44453706,
   'score': 149,
   'total_reactions_count': 104},
  'posts': [{'link': 'https://www.cnn.com/2020/03/06/africa/agriculture-regenerative-farming-climate-crisis-intl/',
    'message': 'These farmers say their cows can solve the climate crisis',
    'name': 'All news, US and international.',
    'post_url': 'https://redd.it/fet09t',
    'profile_image_url': 'https://a.thumbs.redditmedia.com/BCPcjmQTYwA3TAWllW7AJPixfTMQlMxn-fbD2pgcHT4.png',
    'type_id': 11,
    'post_date': '2020-03-07T09:36:44',
    'page_size': 20004441,
    'shares': 0,
    'comments': 24,
    'likes': 40,
    'retweets': 0,
    'love_count': 0,
    'haha_count': 0,
    'wow_count': 0,
    'sad_count': 0,
    'angry_count': 0,
    'thankful_count': 0,
    'total_re

# By Comm/Subscriber

In [202]:
records = []

def create_record(_id, raw_crowd_tangle):
    posts = raw_crowd_tangle['result']['posts']['posts']
    world_news = [x for x in posts if x['name'] == 'World News']
    if world_news:
        world_news = world_news[0]
        others = [x['comments'] / x['page_size'] for x in posts
                  if x['name'] != 'World News'
                  and x['page_size'] > 100000
                  and pd.to_datetime(x['post_date']) < pd.to_datetime(world_news['post_date'])
                 ]
        records.append((_id, world_news['comments'], np.mean(others)))
    else:
        _='foobar'
        
valid_resp.apply(lambda row: create_record(row['_id'], row['reddit_crowd_tangle']), axis=1)
corr_df = pd.DataFrame.from_records(records, columns=['_id', 'num_comments', 'other_comments_per_subscriber'])
corr_df.corr()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,num_comments,other_comments_per_subscriber
num_comments,1.000000,0.330073
other_comments_per_subscriber,0.330073,1.000000


# By Total Comments

In [172]:
records = []

def create_record(_id, raw_crowd_tangle):
    posts = raw_crowd_tangle['result']['posts']['posts']
    world_news = [x for x in posts if x['name'] == 'World News']
    if world_news:
        world_news = world_news[0]
        others = [x['comments'] for x in posts
                  if x['name'] != 'World News'
                  and x['page_size'] > 100000
                  and pd.to_datetime(x['post_date']) < pd.to_datetime(world_news['post_date'])
                 ]
        records.append((_id, world_news['comments'], np.sum(others)))
    else:
        _ = 'foobar'
        
valid_resp.apply(lambda row: create_record(row['_id'], row['reddit_crowd_tangle']), axis=1)
corr_df = pd.DataFrame.from_records(records, columns=['_id', 'num_comments', 'other_comments_per_subscriber'])
corr_df.corr()

,num_comments,other_comments_per_subscriber
num_comments,1.00000,0.00534
other_comments_per_subscriber,0.00534,1.00000


# Model

In [199]:
records = []

def create_record(_id, url, raw_crowd_tangle):
    posts = raw_crowd_tangle['result']['posts']['posts']
    world_news = [x for x in posts if x['name'] == 'World News']
    if world_news:
        world_news = world_news[0]
        others = [(x['comments'], x['name']) for x in posts
                  if x['name'] != 'World News'
                  and x['page_size'] > 1000
                  and pd.to_datetime(x['post_date']) < pd.to_datetime(world_news['post_date'])
                 ]
        for c, n in others:
            records.append((_id, url, world_news['comments'], c, n))
    else:
        _ = 'foobar'
        
valid_resp.apply(lambda row: create_record(row['_id'], row['url'], row['reddit_crowd_tangle']), axis=1)
raw = pd.DataFrame.from_records(records, columns=['_id', 'url', 'num_comments', 'other_comments', 'other_sub'])
corr_df = raw.sort_values('num_comments', ascending=False).drop_duplicates(subset=['url', 'other_sub'], keep='first')
corr_df = corr_df.pivot(index='_id', columns='other_sub', values='other_comments')
# corr_df = corr_df.fillna(-99)
# corr_df = corr_df.merge(raw.groupby('_id').num_comments.agg('first'), on='_id')

In [188]:
corr_df.corr().num_comments.sort_values().tail(50)

FoxFiction                                                                                       0.003219
Interesting content from Spain                                                                   0.003219
Feeds from several unbiased technology sources                                                   0.003762
News Feeds from selected sources                                                                 0.005617
A place for a rational discussion on a divisive topic.                                           0.006487
Australian Politics                                                                              0.007534
Urban Planning: The built environment                                                            0.009896
News from (and relevant to) the UK                                                               0.010705
DeFranco                                                                                         0.010823
Democratic Socialists of America              

In [197]:
predictive_other_subs = [
    'FoxFiction',
'Interesting content from Spain',
'Feeds from several unbiased technology sources',
'News Feeds from selected sources',
'A place for a rational discussion on a divisive topic.',
'Australian Politics',
'Urban Planning: The built environment',
'News from (and relevant to) the UK',
'DeFranco',
# 'Democratic Socialists of America.',
'metacanada',
'Solarpunk - hope for the future',
'Vegan Circlejerk',
"Too long; didn't read (TLDR or TL;DR)",
'SandersForPresident',
'autotldr',
'All things eco',
'The syndicalist commune of ideas',
'Socialism',
'as deep as the world is wide',
'Vote Blue - Vote for Democrats!',
'Tuesday: Subreddit for the Center Right!',
'Hacker News',
'Active Measures',
'r/de - Extraordinär gut!',
'/r/worldpolitics/',
'newsbotBOT',
# 'youtube, but good.',
'Collapse of Civilization',
'Rainbow Capitalism!',
"Reddit's number one hub for the web's most hardened Nazis, Brexiteers, Fascists, and Ukippers",
'Politics without suppression',
'True Patriotism Renewed.',
'Wisconsin: News from the Badger State',
'environment',
'The Grand Tour',
'/r/Vegan - the largest vegan community on the internet!',
    'Feel The Bern',
    'Canada Politics',
#     'FoodForThought: Intellectual Nourishment',
    'News From Around The World',
    'conservatives',
    'On Guard For Thee',
    'Canada',
    'Exceptional Worldwide Weather & Anomalies of Nature',
    'Politics',
    'MarchAgainstTrump',
    "It's Mueller Time!",
    'CanadaLeft',
]

In [200]:
corr_df[predictive_other_subs].info()

<class 'pandas.core.frame.DataFrame'>
Index: 797 entries, 5e65d004bd71f87e9891c891 to 5e65d00bbd71f87e9891d176
Data columns (total 46 columns):
 #   Column                                                                                         Non-Null Count  Dtype  
---  ------                                                                                         --------------  -----  
 0   FoxFiction                                                                                     1 non-null      float64
 1   Interesting content from Spain                                                                 1 non-null      float64
 2   Feeds from several unbiased technology sources                                                 1 non-null      float64
 3   News Feeds from selected sources                                                               3 non-null      float64
 4   A place for a rational discussion on a divisive topic.                                         20 non-null 